# Making the trainig sample from CANDELS GOODS-S 

By Shooby, last edited May29 2019



In [1]:
import numpy as np
import astropy.io.fits as pyfits
import matplotlib.pyplot as plt
import astropy.wcs as wcs
from astropy import units as u
from astropy.coordinates import SkyCoord
from scipy import ndimage
from PIL import Image


%matplotlib inline

In [2]:
gs = pyfits.getdata('../../WFIRST_WPS/CANDELS_fits/gds.fits')
gn = pyfits.getdata('../../WFIRST_WPS/CANDELS_fits/gdn.fits')


In [3]:
sel1 = (gs['zbest']>0.02)&(gs['zbest']<2.)&(gs['CLASS_STAR']<0.85)&(gs['Hmag']<24)&(gs['B_IMAGE_1']/gs['A_IMAGE_1']>0.75)
sel2 = (gn['zbest']>0.02)&(gn['zbest']<2.)&(gn['CLASS_STAR']<0.85)&(gn['Hmag']<24)&(gn['B_IMAGE']/gn['A_IMAGE']>0.75)

sel11 = (gs['zbest']>0.02)&(gs['zbest']<2.)&(gs['CLASS_STAR']<0.85)&(gs['Hmag']<24)&(gs['B_IMAGE_1']/gs['A_IMAGE_1']<0.75)
sel21 = (gn['zbest']>0.02)&(gn['zbest']<2.)&(gn['CLASS_STAR']<0.85)&(gn['Hmag']<24)&(gn['B_IMAGE']/gn['A_IMAGE']<0.75)

print (len(gs[sel1]),len(gn[sel2]),len(gs[sel11]),len(gn[sel21]))


2377 2878 2561 2953


In [5]:
def radec2xy(ra,dec,wc):
    coords = SkyCoord(ra,dec, unit='deg')
    a=wcs.utils.skycoord_to_pixel(coords, wc, origin=0,mode=u'wcs')
    return a[0],a[1]
    
def cut(ra,dec,andaze,filename):
    '''gets coordinates of the galaxy and the filter to return a cutout
    (also called a postage stamp) of the galaxy with given size'''
    hdr = pyfits.getheader(filename)
    w = wcs.WCS(hdr)
    x,y=radec2xy(ra,dec,w)
    x,y=np.int(x),np.int(y)
    im=pyfits.getdata(filename)[y-andaze:y+andaze,x-andaze:x+andaze]
    return im

def segment(a1,above_mean=1.1,add_image_back=0.3,random_noise=0.2):
    '''Returns segmentation of an image by some filtering of lower level 
    noises, to restrict the fitting to the galaxy itself and mask the 
    surroundings'''
    im2=ndimage.gaussian_filter(a1, 2)
    mask = (im2 > above_mean*im2.mean()).astype(np.float)
    mask += add_image_back * im2
    img = mask + random_noise*np.random.randn(*mask.shape)
    hist, bin_edges = np.histogram(img, bins=60)
    bin_centers = 0.5*(bin_edges[:-1] + bin_edges[1:])
    binary_im = img > random_noise
    open_img = ndimage.binary_opening(binary_im)
    close_img = ndimage.binary_closing(open_img)
    return close_img

In [8]:
ra1,dec1=gs['RA_1'][sel1],gs['DEC_1'][sel1]
im1 = cut(ra1[20],dec1[20],32,'/Users/shemmati/Desktop/GOODS/goodss_all_wfc3_ir_f160w_060mas_v1.0_drz.fits')
pyfits.writeto('gal_cutout.fits',im1,overwrite='True')

In [6]:
ra1,dec1=gs['RA_1'][sel1],gs['DEC_1'][sel1]
ra2,dec2=gn['RA_1'][sel2],gn['DEC_1'][sel2]

ra11,dec11=gs['RA_1'][sel11],gs['DEC_1'][sel11]
ra21,dec21=gn['RA_1'][sel21],gn['DEC_1'][sel21]

im = np.zeros([40,40])
for boz in range(2000):
    data = cut(ra1[boz],dec1[boz],20,'/Users/shemmati/Desktop/GOODS/goodss_all_wfc3_ir_f160w_060mas_v1.0_drz.fits')
    data = np.tanh(data)
    rescaled = (255.0 / (data.max()+0.1) * (data - data.min())).astype(np.uint8)
    im = Image.fromarray(rescaled)
    im.save('images/training_images/1/'+str(boz)+'.jpg')
    
    data = cut(ra11[boz],dec11[boz],20,'/Users/shemmati/Desktop/GOODS/goodss_all_wfc3_ir_f160w_060mas_v1.0_drz.fits')
    rescaled = (255.0 / (data.max()+0.1) * (data - data.min())).astype(np.uint8)
    im = Image.fromarray(rescaled)
    im.save('images/training_images/2/'+str(boz)+'.jpg')

for boz in range(2000):
    data = cut(ra2[boz],dec2[boz],20,'/Users/shemmati/Downloads/goodsn_all_wfc3_ir_f160w_060mas_v1.0_drz.fits')
    data = np.tanh(data)
    rescaled = (255.0 / (data.max()+0.1) * (data - data.min())).astype(np.uint8)
    im = Image.fromarray(rescaled)
    im.save('images/training_images/1/'+str(boz)+'_n.jpg')
    
    data = cut(ra21[boz],dec21[boz],20,'/Users/shemmati/Downloads/goodsn_all_wfc3_ir_f160w_060mas_v1.0_drz.fits')
    rescaled = (255.0 / (data.max()+0.1) * (data - data.min())).astype(np.uint8)
    im = Image.fromarray(rescaled)
    im.save('images/training_images/2/'+str(boz)+'_n.jpg')

    #stop
    

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
INFO: 
             

KeyboardInterrupt: 

In [7]:
im = np.zeros([40,40])
for boz in range(2000,2300):
    data = cut(ra1[boz],dec1[boz],20,'/Users/shemmati/Desktop/GOODS/goodss_all_wfc3_ir_f160w_060mas_v1.0_drz.fits')
    data = np.tanh(data)
    rescaled = (255.0 / (data.max()+0.1) * (data - data.min())).astype(np.uint8)
    im = Image.fromarray(rescaled)
    im.save('images/test_images/1/'+str(boz)+'.jpg')
    
    data = cut(ra11[boz],dec11[boz],20,'/Users/shemmati/Desktop/GOODS/goodss_all_wfc3_ir_f160w_060mas_v1.0_drz.fits')
    rescaled = (255.0 / (data.max()+0.1) * (data - data.min())).astype(np.uint8)
    im = Image.fromarray(rescaled)
    im.save('images/test_images/2/'+str(boz)+'.jpg')

for boz in range(2000,2300):
    data = cut(ra2[boz],dec2[boz],20,'/Users/shemmati/Downloads/goodsn_all_wfc3_ir_f160w_060mas_v1.0_drz.fits')
    data = np.tanh(data)
    rescaled = (255.0 / (data.max()+0.1) * (data - data.min())).astype(np.uint8)
    im = Image.fromarray(rescaled)
    im.save('images/test_images/1/'+str(boz)+'_n.jpg')
    
    data = cut(ra21[boz],dec21[boz],20,'/Users/shemmati/Downloads/goodsn_all_wfc3_ir_f160w_060mas_v1.0_drz.fits')
    rescaled = (255.0 / (data.max()+0.1) * (data - data.min())).astype(np.uint8)
    im = Image.fromarray(rescaled)
    im.save('images/test_images/2/'+str(boz)+'_n.jpg')

    #stop
 

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
INFO: 
             